# 무xx 크롤링

## Import package

In [1]:
import urllib
from selenium import webdriver
import time
import pandas as pd
import sys
import os
from tqdm import tqdm

## file name rule
- {number_ls[k]:02d}_\{page:03d}_\{i:02d}.jpg
- number_ls
    - 01 : 후드 집업
    - 02 : 셔츠/블라우스
    - 03 : 긴팔 티셔츠
    - 04 : 맨투맨/스웨트셔츠
    - 05 : 슈트/블레이저 재킷
    - 06 : 후드 티셔츠
    - 07 : 니트/스웨터
    - 08 : 데님 팬츠
    - 09 : 트레이닝/조거 팬츠
    - 10 : 슈트 팬츠/슬랙스
- page : site page number
- i : image index per page

## page information

In [2]:
number_ls = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
page_count = [27, 147, 101, 245, 25, 153, 77, 69, 112, 49]
page_category = ['002022', '001002', '001010', '001005', '002003', '001004', '001006', '003002', '003004', '003008']

## Crawling Code

In [3]:
df = pd.DataFrame(columns=['img', 'title', 'price', 'img_url', 'item_url'])
df.index.name = 'ID'
item_url_src = 'https://store.musinsa.com/app/goods'

for k in range(3, 4):
    for page in tqdm(range(1, page_count[k]+1)):
        binary = './chromedriver/chromedriver.exe'
        driver = webdriver.Chrome(binary)

        # 인기탭 창 띄우기
        # 후드 집업
        url = f"https://search.musinsa.com/category/{page_category[k]}?d_cat_cd={page_category[k]}&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={page}&display_cnt=90&sale_goods=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure="
        driver.get(url)
        time.sleep(2)

        # 남성 항목 클릭
        btn = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[12]/button[2]')
        btn.click()
        time.sleep(1)
        #pattern
        musinsa = driver.find_elements_by_xpath('//img[@class="lazyload lazy"]')

        for i in range(len(musinsa)):

            img_ele = musinsa[i]
            driver.execute_script("arguments[0].scrollIntoView();", img_ele)
            img = img_ele.get_attribute('src')

            img_file = f"../data/image/mall_image/{number_ls[k]:02d}_{page:03d}_{i:02d}.jpg"
            urllib.request.urlretrieve(img, img_file)
            ele = img_ele.find_element_by_xpath('./../../..')
            name = ele.find_element_by_xpath('./div/p[@class="list_info"]').text
            price = ele.find_element_by_xpath('./div/p[@class="price"]').text
#             print(img)
            try: 
                item_url = f"{item_url_src}/{img.split('/')[6]}"
            except:
                item_rul = f"{item_url_src}/{img.split('/')[4]}"
            
            try :
                del_price = ele.find_element_by_xpath('./div/p[@class="price"]/del').text
                price = price.replace(del_price,"")
            except :
                pass

            df.loc[len(df)] = [img_file, name, price, img, item_url]
            if os.path.exists(f"../data/csv/musin.csv"):
                os.remove(f"../data/csv/musin.csv")
                df.to_csv(f'../data/csv/musin.csv', encoding='utf-8')
            else:
                df.to_csv(f'../data/csv/musin.csv', encoding='utf-8')
                
        driver.close()
    df.to_csv(f'../data/csv/mu{number_ls[k]:02d}.csv', encoding='utf-8')
            
if os.path.exists(f'../data/csv/mu_shop.csv') :
    os.remove(f'../data/csv/mu_shop.csv')
else : 
    df.to_csv(f'../data/csv/mu_shop.csv', encoding='utf-8')

100%|██████████| 245/245 [2:23:25<00:00, 35.12s/it]  


In [2]:
df1 = pd.read_csv('../data/csv/mu03.csv')
df2 = pd.read_csv('../data/csv/mu04.csv')
df3 = pd.read_csv('../data/csv/mu07.csv')
df4 = pd.read_csv('../data/csv/mu10.csv')

In [3]:
final_df = pd.concat([df1, df2, df3, df4])

In [4]:
final_df.rename(columns = {'ID':'ID_number'}, inplace=True)

In [11]:
for i in range(final_df.shape[0]):
    final_df['ID_number'].iloc[i] = i

C:\Users\JAY\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [27]:
final_df.set_index(keys=np.arange(len(final_df)), inplace=True)

In [32]:
# final_df.to_csv('../data/csv/FinalMu.csv', encoding='utf-8')
final_df.to_csv('../data/csv/FinalMu.csv', encoding='utf-8')

In [13]:
df_ = pd.read_csv('../data/csv/FinalMu.csv')

In [14]:
df_.drop(columns='Unnamed: 0', inplace=True)

In [15]:
df.set_index

RangeIndex(start=0, stop=89930, step=1)

In [10]:
df_.to_csv('../data/csv/FinalMu.csv', encoding='utf-8')

In [18]:
df_.set_index('ID_number', inplace=True)

In [20]:
df_.to_csv('../data/csv/FinalMu.csv', encoding='utf-8')

In [9]:
df = pd.read_csv('../data/csv/FinalMu.csv')
ls = []
for i in range(df.shape[0]):
    ls.append(df['img'].iloc[i].split('/')[-1])

In [6]:
df['img']

0        ../data/image/mall_image/01_001_00.jpg
1        ../data/image/mall_image/01_001_01.jpg
2        ../data/image/mall_image/01_001_02.jpg
3        ../data/image/mall_image/01_001_03.jpg
4        ../data/image/mall_image/01_001_04.jpg
                          ...                  
89925    ../data/image/mall_image/10_049_14.jpg
89926    ../data/image/mall_image/10_049_15.jpg
89927    ../data/image/mall_image/10_049_16.jpg
89928    ../data/image/mall_image/10_049_17.jpg
89929    ../data/image/mall_image/10_049_18.jpg
Name: img, Length: 89930, dtype: object

In [50]:
ls2 = os.listdir('../data/image/mall_image')

In [25]:
dd = pd.DataFrame(columns=['name', 'bool'])
dd.head()
for j in tqdm(ls):
    for k in range(df.shape[0]):
        if j == ls2[k]:
            dd.loc[len(dd)] = [j, 'True']
            
            
#     len(ls), len(ls2)

100%|██████████| 89930/89930 [16:04<00:00, 93.22it/s] 


In [48]:
len(ls), len(ls2)

(89930, 89930)

In [51]:
data = {'csv':ls, 'imagefile':ls2}
table = pd.DataFrame(data)

In [52]:
for i in tqdm(range(len(ls))):
    if table['csv'].iloc[i] != table['imagefile'].iloc[i]:
        print(i, table['csv'].iloc[i])

100%|██████████| 89930/89930 [00:01<00:00, 87385.02it/s]
